### Create a notebook to perform Runge-Kutta for multiple coupled variables

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

### Define our coupled derivatives to integrate

In [ ]:
def dydx(x,y):
    #set the derivatives
    #our eq is d^2y/dx^2 = -y
    #so write
    #dydx = z
    #dzdx = -y
    
    #we will set y = y[0]
    #we will set z = y[1]
    
    #declare an array, size 2
    y_derivs = np.zeros(2)
    
    #set dydx = z
    y_derivs[0] = y[1]
    
    #set dzdx = -y
    y_derivs[1] = -1*y[0]
    
    #return an array
    return y_derivs

### Def. the 4th order RK method

In [ ]:
def rk4_mv_core(dydx, xi, yi, nv, h):
    
    #declare k? arrays
    #h = step size
    #yi = array
    #nv = number of variables

    #each new k depends on the previous k's, K4 depends on k3 to k1
    #nv in our case is 2, y and z
    k1 = np.zeros(nv)
    k2 = np.zeros(nv)
    k3 = np.zeros(nv)
    k4 = np.zeros(nv)
    k5 = np.zeros(nv)
    k6 = np.zeros(nv)
    

    y_temp = np.zeros(nv)
    
    
    
    #k1 Values
    y_derivs = dydx(xi,yi)
    k1[:] = h*y_derivs[:]
    
    #k2 values
    y_temp[:] = yi[:] + (0.2*k1[:])
    x_temp = xi + (.2*h)
    y_derivs = dydx( x_temp , y_temp )
    k2[:] = h*y_derivs[:]
    
    #get k3 values
    y_temp[:] =yi[:] + (3/40)*k1[:] + ((9/40)*k2[:])
    x_temp = xi + ((3/10)*h)
    y_derivs = dydx(x_temp,y_temp)
    k3[:] = h*y_derivs[:]
    
    #get k4 values
    y_temp[:] = yi[:] + (3/10)*k1[:] + ((-9/10)*k2[:]) + ((6/5)*k3[:])
    x_temp = xi + ((3/5)*h)
    y_derivs = dydx( x_temp , y_temp )
    k4[:] = h * y_derivs[:]

    #get k5 values
    y_temp[:] = yi[:] + ((-11/54)*k1[:]) + ((5/2)*k2[:]) + ((-70/27)*k3[:]) + ((35/27)*k4[:])              
    x_temp = xi + (h)                
    y_derivs = dydx(x_temp, y_temp)         
    k5[:] = h * y_derivs[:]
    
    #get k6 values
    y_temp[:] = yi[:] + ((1631/55296)*k1[:]) + ((175/512)*k2[:]) + ((575/13824)*k3[:]) + ((44275/110592)*k4[:]) + ((253/4096)*k5[:])              
    x_temp = xi + ((7/8)*h)                              
    y_derivs = dydx(x_temp, y_temp)         
    k6[:] = h * y_derivs[:]               
      

    
    #advance y by a step h
    yipo = yi+ ((37/378)*k1)   + 0*k2 + (250/621)*k3 + ((125/594)*k4) + ((0)*k5) + ((512/1771)*k6)
                    
    yipo_star = yi + ((2825/27648)*k1) + 0*k2 + ((18575/48384)*k3) + ((13525/55296)*k4) +  ((277/14336)*k5) + ((.25)*k6)               
    
    Delta = np.fabs(yipo -  yipo_star)
                    
    return yipo, Delta

### Def an adaptive step size driver for RK4


In [ ]:
def rk4_mv_ad(dydx, x_i, y_i, nv, h, tol):
    
    #define safety scale
    #used to estimate an error, 1 big step, 2 little steps
    #if error is bigger than tol, reduce size of step
    #if error is smaller than tol, increase size of step, limit in 
    #increase of step size, half of previous step
    SAFETY = 0.9
    H_NEW_FAC = 2.0
    
    #set a maximum number of iterations
    imax = 10000
    
    #set an iteration variable
    
    i = 0
    
    
    #remember the step
    h_step = h
    
    yipo, Delta = rk4_mv_core(dydx, x_i, y_i, nv, h_step)
    
    
    #adjust step
    while(Delta.max()/tol > 1.0):
            #our error is too large, decrease the step
            h_step *= SAFETY * (Delta.max()/tol)**(-0.25)
            
            yipo, Delta = rk4_mv_core(dydx, x_i, y_i, nv, h_step)    
            #check iteration
            
            if(i>imax):
                print("Too many iterations in rk4_mv_ad()")
                raise StopIteration("Ending after i = ", i)
                
            #iterate 
            i += 1
            
    #next time, try to take a bigger step
    #can only take a step thats 2x as big as current step, due to h_new_fac
    h_new = np. fmin(h_step * (Delta.max()/tol)**(-0.9), h_step*H_NEW_FAC)
    
    #return the answer, a new step, and the step we actually took
    return yipo, h_new, h_step
                
            

In [ ]:
def rk4_mv(dydx, a, b, y_a, tol):
    #dfdx is the derivative wrt x
    #a is the lower bound
    #b is the upper bound
    #y_a are the boundary conditions
    #tol is the tolerane for integrating y
    
    #define our starting step
    xi = a
    yi = y_a.copy()
    
    #an initial step size == make very small
    h = 1.0e-4 *(b-a)
    
    #set a max number of iterations
    imax = 10000
    
    #set an iteration variable
    i = 0
    
    #set the number of coupled odes to the 
    #size of y_a
    nv = len(y_a)
    
    #set the initial conditions
    #single element array with a as that value
    x = np.full(1,a)
    y = np.full((1,nv), y_a)
    
    #set a flag, loop until we reach right side
    #number of steps unkown
    flag = 1
    
    #loop until we reach the right side
    while(flag):
        
        #calculate y_i+1
        yi_new, h_new, h_step = rk4_mv_ad(dydx, xi, yi, nv, h, tol)
        
        #update the step
        h = h_new
        
        #prevent an overshoot
        if(xi+h_step>b):
            
            #take a smaller step
            h = b-xi
            
            #recalculate y_i+1
            yi_new, h_new, h_step = rk4_mv_ad(dydx, xi, yi, nv, h, tol)
            
            #break
            flag = 0
        
        #update values
        xi += h_step
        yi[:] = yi_new[:]
        
        #add the step to the arrays
        x =np.append(x, xi)
        #give it an array and number it'll append xi, and overwrite x with the new array
        
        y_new = np.zeros((len(x),nv))
        y_new[0:len(x)-1,:] = y
            #x-1 because we added an extra element to x, but y and x have to be the same length
        y_new[-1,:] = yi[:]
        del y
        #erases last y matrix
        y = y_new
        
        #prevent too many iterations
        if(i>=imax):
            print("Maximum iterations reached.")
            raise StopIteration("Iteration number= ", i)
            
        #iterate
        i += 1
        
        #output some information
        #3 means a field with 3 elements, 3 digits
        #d means integer
        # \t means print tab followed by x =
        # %9.8 means print out 9 total digits where 8 are on the RHS of decimal
        s = "i = %3d\tx = %9.8f\th = %9.8f\tb=%9.8f" % (i, xi, h_step, b)
        print(s)
        
        #break if new xi is == b
        #ends integration
        if(xi == b):
            flag = 0
    
    #return answer
    return x,y

#### Perform the integration

In [ ]:
a = 0.0
b = 2.0 *np.pi

y_0 = np.zeros(2)
y_0 [0] = 0.0
y_0 [1] = 1.0
nv = 2

tolerance = 1.0e-6

#perform the integration
x, y = rk4_mv(dydx, a, b, y_0, tolerance)

#### Plot the result

In [ ]:
plt.plot(x,y[:,0], 'o', label = 'y(x)')
plt.plot(x, y[:,1], 'o', label='dydx(x)')
xx = np.linspace(0, 2.0*np.pi, 1000)
plt.plot(xx, np.sin(xx), label='sin(x)')
plt.plot(xx, np.cos(xx), label= 'cos(x)')
plt.xlabel('x')
plt.ylabel('y, dy/dx')
plt.legend(frameon=False)


#### Plot error

In [ ]:
sine = np.sin(x)
cosine = np.cos(x)

y_error = (y[:,0]-sine)
dydx_error = (y[:,1]-cosine)

plt.plot(x, y_error, label="y(x) Error")
plt.plot(x, dydx_error, label="y(x) Error")
plt.legend(frameon=False)